## 한국어 전처리 기능


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install soynlp
!pip install konlpy
!pip install soyspacing

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-oeompyng
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-oeompyng
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=dceb852c2de10993a17b4053fd33282174c615b9f809d3f4a65fe4893776cc87
  Stored in directory: /tmp/pip-ephem-wheel-cache-yvk4ly9n/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell
     |████████████████████████████████| 416 kB 5.2 MB/s 
     |████████████████████████████████| 19.4 MB 491 kB/s 
     |████████████████████████████████| 448 kB 51.5 MB/s 


In [ ]:
from hanspell import spell_checker 
import pandas as pd 
import re
from soynlp.normalizer import *
from soyspacing.countbase import CountSpace
from konlpy.tag import Okt
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

In [ ]:
positive_word = [] #긍정
negative_word = [] #부정
toicx_word = [] #욕설

stop_word = []  #불용어

In [ ]:
with open('/content/drive/MyDrive/capstone_design/DATA/positive/positive.txt','r') as file:
  for word in file.readlines():
    word = word.replace('\n','')
    positive_word.append(word)

with open('/content/drive/MyDrive/capstone_design/DATA/negative/negative.txt','r') as file:
  for word in file.readlines():
    word = word.replace('\n','')
    negative_word.append(word)

with open('/content/drive/MyDrive/capstone_design/DATA/toicx/fword_list.txt','r') as file:
  for word in file.readlines():
    word = word.replace('\n','')
    toicx_word.append(word)    


In [ ]:
print(positive_word)
print(negative_word)
print(toicx_word)

['가뿐하다', '감개무량하다', '감격하다', '감동하다', '감미롭다', '감복하다', '감사하다', '감회', '감흥', '경쾌하다', '고맙다', '근사하다', '기분좋다', '기쁘다', '까르르', '깔깔깔깔', '껄껄껄', '끼득끼득', '낄낄낄', '낭만적', '달갑다', '대견하다', '두근거리다', '따뜻하다', '따스하다', '만족하다', '반갑다', '방그레', '방글방글', '방긋', '보기좋다', '빙그레', '뿌듯하다', '사랑스럽다', '상쾌하다', '상큼하다', '설레다', '속시원하다', '신나다', '신명나다', '신바람나다', '싱글벙글하다', '쌩끗', '씰룩거리다', '아하하', '어여쁘다', '얼씨구', '예쁘다', '우아하다', '웃다', '웃어넘기다', '원더풀', '유쾌하다', '으쓱하다', '으하하하', '이힛힛힛', '자부하다', '재미있다', '좋다', '즐겁다', '천만다행이다', '친근하다', '친숙하다', '카핫핫핫', '콩닥거리다', '쾌적하다', '킬킬거리다', '통쾌하다', '편안하다', '하하', '함박웃음', '행복하다', '허허', '후련하다', '후후', '훌륭하다', '흐뭇하다', '흐흐', '흡족하다', '흥겹다', '희희낙락하다', '히죽거리다', '히죽거리다', '히히']
['가관이다', '가련하다', '가소롭다', '가슴아프다', '가슴앓이', '가엾다', '각박하다', '갈구하다', '갈기갈기', '갈망하다', '감질나다', '갑갑하다', '거북하다', '걱정하다', '겁나다', '겁쟁이', '격노하다', '격분하다', '격앙되다', '격양되다', '격해지다', '겸연쩍다', '경박하다', '경악하다', '고달프다', '고독하다', '고리타분하다', '고통스럽다', '곤욕스럽다', '골치아프다', '공포감', '공허하다', '괘씸하다', '괴롭다', '괴상하다', '괴팍하다', '구슬프다', '구역질나다', '구질구질하다.', '굴욕감', '귀찮다', '그리워하다', '근심스럽다',

##전처리 과정 

### Stop_word  정의
무엇을 불용어로 정의해야 하는가?
->  의미 없는 말을 불용어로 해야 한다 

- 나는 행복 하게 살고 싶다.
 

In [ ]:
class Preprocessing:
  def __init__(self,path):
    self.okt = Okt()
    self.sentence_df = pd.read_csv(path+'/final_data.csv',index_col = 0) #24000 개 가량의 수집 데이터 -> pd 형태로 읽어 오기 
    
    self.emoticons_pattern = re.compile("[" #ios, android ,그외의 이모티콘 의 unicode-> 이모티콘 제거를 위한 패턴 을 정규 표현식을 사용하여 compile 
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags=re.UNICODE)
    self.special_characters = [] #특수 문자 , 반복 문자 저장 
    self.stop_words = [] #사용하지 않을 불용어 저장 
    
    with open(path+'/특수문자.txt','r') as file: #model 의 필요없는 특수문자 리스트 모음 가져오기 
      for word in file.readlines():
        word = word.replace('\n','')
        self.special_characters.append(word)
    
    with open(path + '/불용어리스트.txt','r') as file: #수작업으로 제작한 불용어 리스트 파일 리스트로 저장
      for word in file.readlines():
        word = word.replace('\n','')
        self.stop_words.append(word)

  def Spacing(self): #스페이스바 처리 함수 
    for i in range(self.sentence_df.shape[0]):
      sentence = self.sentence_df['Sentence'][i]
      tagged = self.okt.pos(sentence) #해당 문장 속 단어의 품사를 태킹 
      corrected_sentence = "" #띄어쓰기 를 보완 한 문장
      for word in tagged: #품사 태깅 한 단어
         if word[1] in ('Josa','PreEomi','Eomi','Suffix','Punctuation'): #word[1] 품사의 종류 가 조사, 어미 ,접사 , 구두점 ,선어말 어미 띄어쓰기를 하지 않는 품사 
           corrected_sentence += word[0]
         else:
           corrected_sentence +=" "+word[0] # 그외에는 띄어쓰기를 해야함 
        
      if corrected_sentence[0] == " ": #첫글자까 띄어쓰기 형태로 나올경우 를 대비 
        corrected_sentence = corrected_sentence[1:] 
        
      self.sentence_df['Sentence'][i] = corrected_sentence    
  def Basic(self): #기초적인 이모티콘 , 특수문자 등을 제거 , 반복되는 단어 (ㅋㅋ,ㅎㅎ)
    for i in range(self.sentence_df.shape[0]):
      sentence = self.sentence_df['Sentence'][i] #해당 문장을 초기화 
      sentence = self.emoticons_pattern.sub(r'',sentence) #정규표현식을 사용하여 문장의 이모티콘을 제거 
      sentence = repeat_normalize(sentence,num_repeats=1) #반복되는 문자 (ex) ㅋㅋㅋㅋ , ㅎㅎㅎㅎ ,ㅠㅠㅠㅠ 를 한글자로 만듬 ) 
      for word in self.special_characters: #특수문자 제거 
        if word in sentence:
          sentence = sentence.replace(word , '') #뜻이 있는 동사 위주의 단어가 정확도의 핵심 이기 때문에 단어에 의미가 없는 초성 문자들 제거
      self.sentence_df['Sentence'][i] = sentence 
  
  def Spell_checker(self): #맞춤법 검사 함수
    for i in range(self.sentence_df.shape[0]):
      sentence = self.sentence_df['Sentence'][i] #문장 초기화
      sentence = spell_checker.check(sentence) #해당 문장의 맞춤법을 검사
      self.sentence_df['Sentence'][i] = sentence.as_dict()['checked'] #맞춤법 검사를 끝난 문장을 Sentence 문장의 다시 저장 

  def Stop_word(self):  #불용어 처리 
    for i in range(self.sentence_df.shape[0]): 
      sentence = self.sentence_df['Sentence'][i] #해당 문장을 가져오고
      for word in self.stop_words:
        sentence = sentence.replace(word,'')
      self.sentence_df['Sentence'][i] = sentence
  
  def data_return(self):
    return self.sentence_df










In [ ]:
data = Preprocessing('/content/drive/MyDrive/capstone_design/DATA')


In [ ]:
data.Spacing()

In [ ]:
data.sentence_df

,Sentence,label
0,홍진영 언니 진짜 연기 잘 해!! 얼굴두 이쁘구 노래두 잘 부르구 끼도 많음~~ 나...,긍정
1,그동안 열심히 앞만 보면서 묵묵히 살아왔는데 건강이 많이 안 좋다고 하네. 열심히 ...,부정
2,나를 떨리게 하나요~ 부터 완전 끌림~^^ 장범준 노래 멋져요~^^,긍정
3,도움이 되었습니다 감사합니다,긍정
4,똥이 묻었노 ㅋㅋㅋㅋ,욕설
...,...,...
24733,활기찬 하루를 보내려고 해도 기운이 없어. 너무 힘들어.,부정
24734,요리를 하다가 칼에 손이 많이 베여서 병원에 갔는데 다행히 인대는 안 다쳤데 얼마나...,긍정
24735,아직 한국 여자랑 결혼 하는 등신이 있냐,욕설
24736,하다하다 데이트 폭력 범을 방송에 소환 하니......,욕설


In [ ]:
data.Basic()

In [ ]:
data.sentence_df

,Sentence,label
0,홍진영 언니 진짜 연기 잘 해 얼굴두 이쁘구 노래두 잘 부르구 끼도 많음 나는 퍽,긍정
1,그동안 열심히 앞만 보면서 묵묵히 살아왔는데 건강이 많이 안 좋다고 하네 열심히 살...,부정
2,나를 떨리게 하나요 부터 완전 끌림 장범준 노래 멋져요,긍정
3,도움이 되었습니다 감사합니다,긍정
4,똥이 묻었노,욕설
...,...,...
24733,활기찬 하루를 보내려고 해도 기운이 없어 너무 힘들어,부정
24734,요리를 하다가 칼에 손이 많이 베여서 병원에 갔는데 다행히 인대는 안 다쳤데 얼마나...,긍정
24735,아직 한국 여자랑 결혼 하는 등신이 있냐,욕설
24736,하다하다 데이트 폭력 범을 방송에 소환 하니,욕설


In [ ]:
data.Stop_word()

In [ ]:
data.sentence_df

,Sentence,label
0,홍진영 언니 연기 잘 해 얼굴두 이쁘구 노래두 잘 부르구 끼도 많음 나는 퍽,긍정
1,그동안 열심히 앞만 보면서 묵묵히 살아왔는데 건강이 많이 안 좋다고 하네 열심히 살...,부정
2,나를 떨리게 하나요 부터 완전 끌림 장범준 노래 멋져요,긍정
3,도움이 되었습니다 감사합니다,긍정
4,똥이 묻었노,욕설
...,...,...
24733,활기찬 하루를 보내려고 해도 기운이 없어 힘들어,부정
24734,요리를 하다가 칼에 손이 많이 베여서 병원에 갔는데 다행히 인대는 안 다쳤데 얼마나...,긍정
24735,아직 한국 여자랑 결혼 하는 등신이 있냐,욕설
24736,하다하다 데이트 폭력 범을 방송에 소환 하니,욕설


In [ ]:
df = data.data_return()

In [ ]:
df

,Sentence,label
0,홍진영 언니 연기 잘 해 얼굴두 이쁘구 노래두 잘 부르구 끼도 많음 나는 퍽,긍정
1,그동안 열심히 앞만 보면서 묵묵히 살아왔는데 건강이 많이 안 좋다고 하네 열심히 살...,부정
2,나를 떨리게 하나요 부터 완전 끌림 장범준 노래 멋져요,긍정
3,도움이 되었습니다 감사합니다,긍정
4,똥이 묻었노,욕설
...,...,...
24733,활기찬 하루를 보내려고 해도 기운이 없어 힘들어,부정
24734,요리를 하다가 칼에 손이 많이 베여서 병원에 갔는데 다행히 인대는 안 다쳤데 얼마나...,긍정
24735,아직 한국 여자랑 결혼 하는 등신이 있냐,욕설
24736,하다하다 데이트 폭력 범을 방송에 소환 하니,욕설


In [ ]:
df.to_csv('data.csv',encoding = 'utf-8-sig')